<a href="https://colab.research.google.com/github/jogarrid/nuscenes/blob/master/vanilla5dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# > IMPLEMENTATION OF VANILLA LSTM MODEL  TO TRAJECTORY DATA IN THE NUSCENES DATASET.
COMPARISON BETWEEN PERFORMANCE (MSE) BETWEEN A LINNEAR PREDICTOR AND THE LSTM PREDICTOR





In [0]:
import numpy as np
import pickle
from google.colab import drive
drive.mount('/content/gdrive')
map_data_split = np.load('/content/gdrive/My Drive/map_data_split.npy')  #scenes x partitions x instances x time_steps x 2

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
f = open('/content/gdrive/My Drive/maps_list.pkl', "rb")
maps_list = pickle.load(f)
f.close()


In [0]:
import os
import pickle
import numpy as np
import random

class DataLoader():

    def __init__(self, batch_size=50, seq_length=5, datasets=list(range(90)), force_preprocessing = True, testing=False):
        '''
        Initialiser function for the DataLoader class
        params:
        batch_size : Size of the mini-batch
        seq_length : RNN sequence length
        '''

        self.used_data_dirs = datasets

        # Data directory where the pre-processed pickle file resides
        self.data_dir = '/content/gdrive/My Drive/vanillaData/'

        # Store the batch size and the sequence length arguments
        self.batch_size = batch_size
        self.seq_length = seq_length

        # Define the path of the file in which the data needs to be stored
        data_file = os.path.join(self.data_dir, "trajectories.cpkl")

        self.testing = testing
        
        # If the file doesn't exist already or if forcePreProcess is true
        if not(os.path.exists(data_file)) or force_preprocessing:
            print("Creating pre-processed data from raw data")
            # Preprocess the data from the csv files
            self.preprocess(self.used_data_dirs, data_file)

        # Load the data from the pickled file
        self.load_preprocessed(data_file)
        # Reset all the pointers
        self.reset_batch_pointer()

    def preprocess(self, scenes, data_file):
        '''
        scenes : List of scene indixes to be loaded
        data_file : The file into which all the pre-processed data needs to be stored
        '''
        # all_ins_data would be a dictionary with mapping from each instance to their
        # trajectories given by matrix 3 x numPoints with each column
        # in the order x, y, frameId

        all_ins_data = {}
        dataset_indices = []
        current_ins = 0
        threshold = 2
        # where each column is a (frameId, instance_Id, y, x) vector
 
        data_all = np.load('/content/gdrive/My Drive/map_data_split.npy')  #scenes x partitions x instances x time_steps x 2
        data_all = data_all[scenes]
        
        frame_Id = 0
        instance_Id = 0
        data = []

        means = []
        stds = []
        indexes = []
        self.mean_std = []
        for scene_ix in range(data_all.shape[0]):
          for partition_ix in range(data_all.shape[1]):
            part = data_all[scene_ix, partition_ix]
            for instance_ix in range(part.shape[0]):
              traj_x = data_all[scene_ix, partition_ix][instance_ix, :, 0]
              traj_y = data_all[scene_ix, partition_ix][instance_ix, :, 1]

              if(~np.any(traj_x == -999) and ~np.any(traj_y == -999)):
                
                #create new instance trajectories by rotating the ones we have
                if(not self.testing):
                  angles = list(np.arange(0,2*np.pi,0.7))
                  traj_x_or = np.copy(traj_x)
                  traj_y_or = np.copy(traj_y)
                  for angle in angles:
                    traj_x = traj_x_or*np.cos(angle)-traj_y_or*np.sin(angle)
                    traj_y = traj_y_or*np.cos(angle)+traj_x_or*np.sin(angle)
                    
                    mean_x = np.mean(traj_x[0:4])
                    mean_y = np.mean(traj_y[0:4])
                    std_x = np.std(traj_x[0:4])
                    std_y = np.std(traj_y[0:4])
                    #if(std_x > threshold or std_y>threshold):
                    traj_x = (traj_x - mean_x)/(std_x+0.5)
                    traj_y = (traj_y - mean_y)/(std_y+0.5)
                    means.append([mean_x, mean_y])
                    stds.append([std_x, std_y])
                    indexes.append([scene_ix, partition_ix, instance_ix])

                    for time_ix in range(traj_x.shape[0]):
                      data.append([frame_Id, instance_Id, traj_x[time_ix], traj_y[time_ix]])
                    self.mean_std.append([[mean_x, std_x], [mean_y, std_y], [scene_ix, partition_ix, instance_ix]])
                    instance_Id += 1 
                    
                else:
                  mean_x = np.mean(traj_x[0:4])
                  mean_y = np.mean(traj_y[0:4])
                  std_x = np.std(traj_x[0:4])
                  std_y = np.std(traj_y[0:4])
                  #if(std_x > threshold or std_y>threshold):
                  traj_x = (traj_x - mean_x)/(std_x+0.5)
                  traj_y = (traj_y - mean_y)/(std_y+0.5)
                  means.append([mean_x, mean_y])
                  stds.append([std_x, std_y])
                  indexes.append([scene_ix, partition_ix, instance_ix])

                  for time_ix in range(traj_x.shape[0]):
                    data.append([frame_Id, instance_Id, traj_x[time_ix], traj_y[time_ix]])
                  self.mean_std.append([[mean_x, std_x], [mean_y, std_y], [scene_ix, partition_ix, instance_ix]])
                  instance_Id += 1    
            frame_Id += 1
        data = np.array(data).T

        # Get the number of pedestrians in the current dataset
        numIns = np.size(np.unique(data[1, :]))  #an instance can be a car, a pedestrian, a bycicle, etc...

        for instance_Id in range(numIns):
            # Extract trajectory of the current instance
            traj = data[:, data[1, :] == instance_Id]
                        
            # Format it as (x, y, frameId)
            traj = traj[[2, 3, 0], :]

            # Store this in the dictionary
            all_ins_data[instance_Id] = traj

            mean_std = self.mean_std[instance_Id]
            mean_x = mean_std[0][0]
            std_x = mean_std[0][1]
            mean_y = mean_std[1][0]
            std_y = mean_std[1][1]
            
            traj_r = np.zeros(traj.shape)
            traj_r[0,:] =  traj[0,:] *(std_x +0.5)+mean_x
            traj_r[1,:] = traj[1,:] *(std_y +0.5)+mean_y
            index = indexes[instance_Id]

        # Current dataset done
        dataset_indices.append(current_ins+numIns)

        # The complete data is a tuple of all intance data, and dataset ped indices
        complete_data = (all_ins_data, dataset_indices)
        # Store the complete data into the pickle file
        f = open(data_file, "wb")
        pickle.dump(complete_data, f, protocol=2)
        f.close()

    def load_preprocessed(self, data_file):
        '''
        Function to load the pre-processed data into the DataLoader object
        params:
        data_file : The path to the pickled data file
        '''
        # Load data from the pickled file
        f = open(data_file, "rb")
        self.raw_data = pickle.load(f)
        f.close()

        # Get the data from the pickle file
        all_ins_data = self.raw_data[0]

        # Construct the data with sequences(or trajectories) longer than seq_length
        self.data = []
        counter = 0

        # For each instance in the data
        
        for instance_Id in all_ins_data:
            # Extract his trajectory
            traj = all_ins_data[instance_Id]
            
            # If the length of the trajectory is greater than seq_length 
            if traj.shape[1] > (self.seq_length):
                self.data.append(traj[[0, 1], :].T)
                counter += 1

        #shuffle the data (if training) so all trajectories belonging to an instance won´t be in the same batch in different epoch
        indexes = np.arange(len(self.data))
        self.data = np.array(self.data)
        self.mean_std = np.array(self.mean_std)
        
        #if training, shuffle the data
        if(not self.testing): 
          np.random.shuffle(indexes)
        
        self.data = self.data[indexes]
        self.mean_std = self.mean_std[indexes]
                
        # Calculate the number of batches (each of batch_size) in the data
        self.num_batches = int(counter / self.batch_size)

        
    def next_batch(self):
        '''
        Function to get the next batch of points
        '''
        # List of source and target data for the current batch
        x_batch = []
        y_batch = []
        # For each sequence in the batch
        for i in range(self.batch_size):
            # Extract the trajectory of the pedestrian pointed out by self.pointer
            traj = self.data[self.pointer]
            mean_std = self.mean_std[self.pointer]
            idx = 0 
            # Append the trajectory from idx until seq_length into source and target data
            x_batch.append(np.copy(traj[idx:idx+self.seq_length, :]))
            y_batch.append(np.copy(traj[idx+1:idx+self.seq_length+1, :]))
            self.tick_batch_pointer()

        return x_batch, y_batch, mean_std

    def tick_batch_pointer(self):
        '''
        Advance the data pointer
        '''
      
        #TODO: randomize data as you set the pointer to 0 so that you don´t train always over the same batches
        self.pointer += 1
        if (self.pointer >= len(self.data)):
            self.pointer = 0
            indexes = np.arange(len(self.data))
            self.data = np.array(self.data)
            self.mean_std = np.array(self.mean_std)

            #if training, shuffle the data
            if(not self.testing): 
              np.random.shuffle(indexes)

            self.data = self.data[indexes]
            self.mean_std = self.mean_std[indexes]

    def reset_batch_pointer(self):
        '''
        Reset the data pointer
        '''
        self.pointer = 0

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops import rnn_cell


# The Vanilla LSTM model
class Model():
    def __init__(self, args, infer=False):
        '''
        I
        nitialisation function for the class Model.
        Params:
        args: Contains arguments required for the Model creation
        '''

        # If sampling new trajectories, then infer mode
        if infer:
            # Infer one position at a time
            args.batch_size = 1
            args.seq_length = 1

        # Store the arguments
        self.args = args

        # Initialize a BasicLSTMCell recurrent unit
        # args.rnn_size contains the dimension of the hidden state of the LSTM
        cell = rnn_cell.BasicLSTMCell(args.rnn_size, state_is_tuple=False)

        # Multi-layer RNN construction, if more than one layer
        cell = rnn_cell.MultiRNNCell([cell] * args.num_layers, state_is_tuple=False)

        # TODO: (improve) Dropout layer can be added here
        # Store the recurrent unit
        self.cell = cell

        # TODO: (resolve) Do we need to use a fixed seq_length?
        # Input data contains sequence of (x,y) points
        self.input_data = tf.placeholder(tf.float32, [None, args.seq_length, 2])
        # target data contains sequences of (x,y) points as well
        self.target_data = tf.placeholder(tf.float32, [None, args.seq_length, 2])

        # Learning rate
        self.lr = tf.Variable(args.learning_rate, trainable=False, name="learning_rate")

        # Initial cell state of the LSTM (initialised with zeros)
        self.initial_state = cell.zero_state(batch_size=args.batch_size, dtype=tf.float32)

        # Output size is the set of parameters (mu, sigma, corr)
        output_size = 5  # 2 mu, 2 sigma and 1 corr

        # Embedding for the spatial coordinates
        with tf.variable_scope("coordinate_embedding"):
            #  The spatial embedding using a ReLU layer
            #  Embed the 2D coordinates into embedding_size dimensions
            #  TODO: (improve) For now assume embedding_size = rnn_size
            embedding_w = tf.get_variable("embedding_w", [2, args.embedding_size])
            embedding_b = tf.get_variable("embedding_b", [args.embedding_size])

        # Output linear layer
        with tf.variable_scope("rnnlm"):
            output_w = tf.get_variable("output_w", [args.rnn_size, output_size], initializer=tf.truncated_normal_initializer(stddev=0.01), trainable=True)
            output_b = tf.get_variable("output_b", [output_size], initializer=tf.constant_initializer(0.01), trainable=True)

        # Split inputs according to sequences.
        inputs = tf.split(self.input_data, args.seq_length, 1)
        # Get a list of 2D tensors. Each of size numPoints x 2
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

        # Embed the input spatial points into the embedding space
        embedded_inputs = []
        for x in inputs:
            # Each x is a 2D tensor of size numPoints x 2
            # Embedding layer
            embedded_x = tf.nn.relu(tf.add(tf.matmul(x, embedding_w), embedding_b))
            embedded_inputs.append(embedded_x)

        # Feed the embedded input data, the initial state of the LSTM cell, the recurrent unit to the seq2seq decoder
        outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(embedded_inputs, self.initial_state, self.cell, loop_function=None)

        # Concatenate the outputs from the RNN decoder and reshape it to ?xargs.rnn_size
        output = tf.reshape(tf.concat( outputs,1), [-1, args.rnn_size])

        # Apply the output linear layer
        output = tf.nn.xw_plus_b(output, output_w, output_b)
        
        # Store the final LSTM cell state after the input data has been feeded
        self.final_state = last_state

        # reshape target data so that it aligns with predictions
        flat_target_data = tf.reshape(self.target_data, [-1, 2])
        # Extract the x-coordinates and y-coordinates from the target data
        [x_data, y_data] = tf.split(flat_target_data, 2, 1)

        def tf_2d_normal(x, y, mux, muy, sx, sy, rho):
            '''
            Function that implements the PDF of a 2D normal distribution
            params:
            x : input x points
            y : input y points
            mux : mean of the distribution in x
            muy : mean of the distribution in y
            sx : std dev of the distribution in x
            sy : std dev of the distribution in y
            rho : Correlation factor of the distribution
            '''
            # eq 3 in the paper
            # and eq 24 & 25 in Graves (2013)
            # Calculate (x - mux) and (y-muy)
            
            normx = tf.subtract(x, mux)
            normy = tf.subtract(y, muy)
            # Calculate sx*sy
            sxsy = tf.multiply(sx, sy)
            # Calculate the exponential factor
            z = tf.square(tf.div(normx, sx)) + tf.square(tf.div(normy, sy)) - 2*tf.div(tf.multiply(rho, tf.multiply(normx, normy)), sxsy)
            negRho = 1 - tf.square(rho)
            # Numerator
            result = tf.exp(tf.div(-z, 2*negRho))
            # Normalization constant
            denom = 2 * np.pi * tf.multiply(sxsy, tf.sqrt(negRho))
            # Final PDF calculation
            result = tf.div(result, denom)
            self.result = result
            return result

        # Important difference between loss func of Social LSTM and Graves (2013)
        # is that it is evaluated over all time steps in the latter whereas it is
        # done from t_obs+1 to t_pred in the former
        
        def get_lossfunc(z_mux, z_muy, z_sx, z_sy, z_corr, x_data, y_data):
            '''
            Function to calculate given a 2D distribution over x and y, and target data
            of observed x and y points
            params:
            z_mux : mean of the distribution in x
            z_muy : mean of the distribution in y
            z_sx : std dev of the distribution in x
            z_sy : std dev of the distribution in y
            z_rho : Correlation factor of the distribution
            x_data : target x points
            y_data : target y points
            '''
            step = tf.constant(1e-3, dtype=tf.float32, shape=(1, 1))

            # Calculate the PDF of the data w.r.t to the distribution
            result0_1 = tf_2d_normal(x_data, y_data, z_mux, z_muy, z_sx, z_sy, z_corr)
            result0_2 = tf_2d_normal(tf.add(x_data, step), y_data, z_mux, z_muy, z_sx, z_sy, z_corr)
            result0_3 = tf_2d_normal(x_data, tf.add(y_data, step), z_mux, z_muy, z_sx, z_sy, z_corr)
            result0_4 = tf_2d_normal(tf.add(x_data, step), tf.add(y_data, step), z_mux, z_muy, z_sx, z_sy, z_corr)

            result0 = tf.div(tf.add(tf.add(tf.add(result0_1, result0_2), result0_3), result0_4), tf.constant(4.0, dtype=tf.float32, shape=(1, 1)))
            result0 = tf.multiply(tf.multiply(result0, step), step)

            # For numerical stability purposes
            epsilon = 1e-20

            # TODO: (resolve) I don't think we need this as we don't have the inner
            # summation
            # result1 = tf.reduce_sum(result0, 1, keep_dims=True)
            # Apply the log operation
            result1 = -tf.log(tf.maximum(result0, epsilon))  # Numerical stability

            # TODO: For now, implementing loss func over all time-steps
            # Sum up all log probabilities for each data point
            return tf.reduce_sum(result1)

        def get_coef(output):
            # eq 20 -> 22 of Graves (2013)
            # TODO : (resolve) Does Social LSTM paper do this as well?
            # the paper says otherwise but this is essential as we cannot
            # have negative standard deviation and correlation needs to be between
            # -1 and 1

            z = output
            # Split the output into 5 parts corresponding to means, std devs and corr
            z_mux, z_muy, z_sx, z_sy, z_corr = tf.split(z, 5, 1)

            # The output must be exponentiated for the std devs
            z_sx = tf.exp(z_sx)
            z_sy = tf.exp(z_sy)
            # Tanh applied to keep it in the range [-1, 1]
            z_corr = tf.tanh(z_corr)

            return [z_mux, z_muy, z_sx, z_sy, z_corr]
        
        # Extract the coef from the output of the linear layer
        [o_mux, o_muy, o_sx, o_sy, o_corr] = get_coef(output)
        # Store the output from the model
        self.output = output

        # Store the predicted outputs
        self.mux = o_mux
        self.muy = o_muy
        self.sx = o_sx
        self.sy = o_sy
        self.corr = o_corr

        # Compute the loss function
        lossfunc = get_lossfunc(o_mux, o_muy, o_sx, o_sy, o_corr, x_data, y_data)

        # Compute the cost
        self.cost = tf.div(lossfunc, (args.batch_size * args.seq_length))

        # Get trainable_variables
        tvars = tf.trainable_variables()

        # TODO: (resolve) We are clipping the gradients as is usually done in LSTM
        # implementations. Social LSTM paper doesn't mention about this at all
        # Calculate gradients of the cost w.r.t all the trainable variables
        self.gradients = tf.gradients(self.cost, tvars)
        # Clip the gradients if they are larger than the value given in args
        grads, _ = tf.clip_by_global_norm(self.gradients, args.grad_clip)

        # NOTE: Using RMSprop as suggested by Social LSTM instead of Adam as Graves(2013) does
        # optimizer = tf.train.AdamOptimizer(self.lr)
        # initialize the optimizer with teh given learning rate
        optimizer = tf.train.RMSPropOptimizer(self.lr)

        # Train operator
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def sample(self, sess, traj, num=10):
        '''
        Given an initial trajectory (as a list of tuples of points), predict the future trajectory
        until a few timesteps
        Params:
        sess: Current session of Tensorflow
        traj: List of past trajectory points
        num: Number of time-steps into the future to be predicted
        '''
        def sample_gaussian_2d(mux, muy, sx, sy, rho):
            '''
            Function to sample a point from a given 2D normal distribution
            params:
            mux : mean of the distribution in x
            muy : mean of the distribution in y
            sx : std dev of the distribution in x
            sy : std dev of the distribution in y
            rho : Correlation factor of the distribution
            '''
            # Extract mean
            mean = [mux, muy]
            # Extract covariance matrix
            cov = [[sx*sx, rho*sx*sy], [rho*sx*sy, sy*sy]]
            # Sample a point from the multivariate normal distribution
            x = np.random.multivariate_normal(mean, cov, 1)
            return x[0][0], x[0][1]

        # Initial state with zeros
        state = sess.run(self.cell.zero_state(1, tf.float32))

        # Iterate over all the positions seen in the trajectory
        for pos in traj[:-1]:
            # Create the input data tensor
            data = np.zeros((1, 1, 2), dtype=np.float32)
            data[0, 0, 0] = pos[0]  # x
            data[0, 0, 1] = pos[1]  # y

            # Create the feed dict
            feed = {self.input_data: data, self.initial_state: state}
            # Get the final state after processing the current position
            [state] = sess.run([self.final_state], feed)

        ret = traj

        # Last position in the observed trajectory
        last_pos = traj[-1]

        # Construct the input data tensor for the last point
        prev_data = np.zeros((1, 1, 2), dtype=np.float32)
        prev_data[0, 0, 0] = last_pos[0]  # x
        prev_data[0, 0, 1] = last_pos[1]  # y

        for t in range(num):
            # Create the feed dict
            feed = {self.input_data: prev_data, self.initial_state: state}

            # Get the final state and also the coef of the distribution of the next point
            [o_mux, o_muy, o_sx, o_sy, o_corr, state] = sess.run([self.mux, self.muy, self.sx, self.sy, self.corr, self.final_state], feed)

            # Sample the next point from the distribution
            next_x, next_y = sample_gaussian_2d(o_mux[0][0], o_muy[0][0], o_sx[0][0], o_sy[0][0], o_corr[0][0])
            # Append the new point to the trajectory
            ret = np.vstack((ret, [next_x, next_y]))

            # Set the current sampled position as the last observed position
            prev_data[0, 0, 0] = next_x
            prev_data[0, 0, 1] = next_y

        return ret
      
      

In [0]:
import numpy as np
import tensorflow as tf
import argparse
import os
import time
import pickle

#things to do : 

#exlude samples with low std (<1) from training (say they are not moving and won´t start moving...)
#adjust hyper param
#Integrate map info

class arguments():
   def __init__(self, rnn_size = 128, num_layers = 5, model = 'lstm', batch_size = 50, seq_length = 9,
         num_epochs = 100, save_every = 500,  grad_clip = 10, learning_rate = 0.003,
         decay_rate = 0.99, keep_prob = 0.7, embedding_size = 128, leaveDataset = 1):
    self.rnn_size = rnn_size
    self.num_layers = num_layers
    self.model = model
    self.batch_size = batch_size
    self.seq_length = seq_length
    self.num_epochs = num_epochs
    self.save_every = save_every
    self.grad_clip = grad_clip
    self.learning_rate = learning_rate
    self.decay_rate = decay_rate
    self.keep_prob = keep_prob
    self.embedding_size = embedding_size
    self.leaveDataset = leaveDataset

def main():
  args = arguments()
  tf.reset_default_graph()#SAMPLE
  train(args)


def train(args):
    datasets = list(range(90)) #we train with first 90 scenes

    # Create the data loader object. This object would preprocess the data in terms of
    # batches each of size args.batch_size, of length args.seq_length
    data_loader = DataLoader(args.batch_size, args.seq_length, datasets, force_preprocessing=True, testing = False)

    # Save the arguments int the config file
    with open(os.path.join('/content/gdrive/My Drive/vanillaData', 'config.pkl'), 'wb') as f:
        pickle.dump(args, f)

    # Create a Vanilla LSTM model with the arguments
    model = Model(args)

    # Initialize a TensorFlow session
    with tf.Session() as sess:
        # Initialize all the variables in the graph
        sess.run(tf.initialize_all_variables())
        # Add all the variables to the list of variables to be saved
        saver = tf.train.Saver(tf.all_variables())

        # For each epoch
        print(args.num_epochs)
        for e in range(args.num_epochs):
            # Assign the learning rate (decayed acc. to the epoch number)
            sess.run(tf.assign(model.lr, args.learning_rate * (args.decay_rate ** e)))
            # Reset the pointers in the data loader object
            data_loader.reset_batch_pointer()
            # Get the initial cell state of the LSTM
            state = sess.run(model.initial_state)

            # For each batch in this epoch
            losst = 0
            for b in range(data_loader.num_batches):
                # Tic
                start = time.time()
                # Get the source and target data of the current batch
                # x has the source data, y has the target data
                x, y, mean_std = data_loader.next_batch()

                # Feed the source, target data and the initial LSTM state to the model
                feed = {model.input_data: x, model.target_data: y, model.initial_state: state}
                # Fetch the loss of the model on this batch, the final LSTM state from the session
                train_loss, state, _ , pred= sess.run([model.cost, model.final_state, model.train_op, model.output], feed)
                                # Toc
                losst += train_loss
                end = time.time()
                if(b==50):
                  print(
                    "{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}"
                    .format(
                        e * data_loader.num_batches + b,
                        args.num_epochs * data_loader.num_batches,
                        e,
                        losst/b, end - start))
                  losst=0

                # Save the model if the current epoch and batch number match the frequency
                if (e * data_loader.num_batches + b) % args.save_every == 0 and ((e * data_loader.num_batches + b) > 0):
                    checkpoint_path = os.path.join('save', 'model.ckpt')
                    saver.save(sess, checkpoint_path, global_step=e * data_loader.num_batches + b)
                    print("model saved to {}".format(checkpoint_path))

main()

Creating pre-processed data from raw data
100
50/96700 (epoch 0), train_loss = 22.108, time/batch = 0.037
model saved to save/model.ckpt
model saved to save/model.ckpt
1017/96700 (epoch 1), train_loss = 17.599, time/batch = 0.038
model saved to save/model.ckpt
1984/96700 (epoch 2), train_loss = 17.472, time/batch = 0.036
model saved to save/model.ckpt
model saved to save/model.ckpt
2951/96700 (epoch 3), train_loss = 16.456, time/batch = 0.036
model saved to save/model.ckpt
model saved to save/model.ckpt
3918/96700 (epoch 4), train_loss = 16.285, time/batch = 0.035
model saved to save/model.ckpt
model saved to save/model.ckpt
4885/96700 (epoch 5), train_loss = 15.561, time/batch = 0.034
model saved to save/model.ckpt
model saved to save/model.ckpt
5852/96700 (epoch 6), train_loss = 15.289, time/batch = 0.035
model saved to save/model.ckpt
model saved to save/model.ckpt
6819/96700 (epoch 7), train_loss = 14.713, time/batch = 0.035
model saved to save/model.ckpt
model saved to save/model.

# Get test MSE using LSTM and a linear predictor


In [0]:
def get_mean_error(predicted_traj, true_traj, observed_length):
    '''
    Function that computes the mean euclidean distance error between the
    predicted and the true trajectory
    params:
    predicted_traj : numpy matrix with the points of the predicted trajectory
    true_traj : numpy matrix with the points of the true trajectory
    observed_length : The length of trajectory observed
    '''
    # The data structure to store all errors
    error = np.zeros(len(true_traj) - observed_length)
    # For each point in the predicted part of the trajectory
    for i in range(observed_length, len(true_traj)):
        # The predicted position
        pred_pos = predicted_traj[i, :]
        # The true position
        true_pos = true_traj[i, :]

        # The euclidean distance is the error
        error[i-observed_length] = np.linalg.norm(true_pos - pred_pos)

    # Return the mean error
    return np.mean(error)

class ParserArgs():
  def __init__(self):
    self.obs_length = 5
    self.pred_length = 4
    self.test_dataset = list(range(90,100))
    

# Read the arguments
sample_args = ParserArgs()

#test scenes
testScenes = list(range(90,100))

# Load the saved arguments to the model from the config file
with open(os.path.join('/content/gdrive/My Drive/vanillaData', 'config.pkl'), 'rb') as f:
    saved_args = pickle.load(f)

#with trainer.model.graph.as_default():
# Initialize TensorFlow session
tf.reset_default_graph()#SAMPLE

# Initialize with the saved args
model = Model(saved_args, True)

sess = tf.InteractiveSession()
# Initialize TensorFlow saver
saver = tf.train.Saver()

# Get the checkpoint state to load the model from
ckpt = tf.train.get_checkpoint_state('save')

print('loading model: ', ckpt.model_checkpoint_path)
# Restore the model at the checpoint
saver.restore(sess, ckpt.model_checkpoint_path)

# Initialize the dataloader object to
# Get sequences of length obs_length+pred_length
data_loader = DataLoader(1, sample_args.pred_length + sample_args.obs_length, testScenes, force_preprocessing= True, testing = True)

mean_std = data_loader.mean_std
# Reset the data pointers of the data loader object
data_loader.reset_batch_pointer()

# Maintain the total_error until now
total_error = 0.
counter = 0.
j = 0

trajs = []
info = [] #mean, std, index information in array form
for b in range(data_loader.num_batches):
    # Get the source, target data for the next batch
    x, y, mean_std = data_loader.next_batch()
    mean_x = mean_std[0][0]
    std_x = mean_std[0][1]
    mean_y = mean_std[1][0]
    std_y = mean_std[1][1]
    traj_real = x[0]
    # The observed part of the trajectory
    obs_traj = x[0][:sample_args.obs_length]
    
    # Get the complete trajectory with both the observed and the predicted part from the model
    complete_traj = model.sample(sess, obs_traj, num=sample_args.pred_length)
    j+=1
    obs_traj = np.array(obs_traj)
    print(obs_traj.shape)
    complete_traj[:,0] = complete_traj[:,0] *(std_x +0.5)+mean_x
    complete_traj[:,1] = complete_traj[:,1] *(std_y +0.5)+mean_y
    
    traj_real[:,0] = traj_real[:,0] *(std_x +0.5)+mean_x
    traj_real[:,1] = traj_real[:,1] *(std_y +0.5)+mean_y
    

    # Compute the mean error between the predicted part and the true trajectory
    error = get_mean_error(complete_traj, traj_real, sample_args.obs_length)
    total_error += error
    print("Processed trajectory number : ", b, "out of ", data_loader.num_batches, " trajectories")
    info.append(mean_std)
    trajs.append(complete_traj)
    index = mean_std[2]
    if(j<50):
      print(mean_x)
      print(complete_traj)
      print(map_data_split[90+index[0], index[1]][index[2]])
      print(error)

# Print the mean error across all the batches
print("Total mean error of the model is ", total_error/data_loader.num_batches)

loading model:  save/model.ckpt-96500
INFO:tensorflow:Restoring parameters from save/model.ckpt-96500


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Creating pre-processed data from raw data
(5, 2)
Processed trajectory number :  0 out of  586  trajectories
3188.0552978536757
[[ 3159.04370528 10474.6001755 ]
 [ 3178.63540151 10429.59645951]
 [ 3197.65260414 10385.365235  ]
 [ 3216.88948048 10341.43685603]
 [ 3235.86855835 10298.28985586]
 [ 3255.7999183  10254.39340298]
 [ 3278.40134082 10206.36530928]
 [ 3298.5811752  10164.33426108]
 [ 3309.31310981 10145.7720501 ]]
[[ 3159.04370528 10474.6001755 ]
 [ 3178.63540151 10429.59645951]
 [ 3197.65260414 10385.365235  ]
 [ 3216.88948048 10341.43685603]
 [ 3235.86855835 10298.28985586]
 [ 3254.10794472 10256.27910802]
 [ 3272.8876781  10214.07059312]
 [ 3291.54139771 10172.55610703]
 [ 3305.94341512 10139.88822281]
 [ 3323.01729004 10099.92607869]]
7.403165147250779
(5, 2)
Processed trajectory number :  1 out of  586  trajectories
3371.5125
[[ 3392.9        10089.47      ]
 [ 3378.61       10119.87      ]
 [ 3363.23       10152.57      ]
 [ 3351.31       10182.29      ]
 [ 3339.25       1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:246: RuntimeWarning: covariance is not positive-semidefinite.


(5, 2)
Processed trajectory number :  4 out of  586  trajectories
3454.0325
[[ 3471.23        9901.52      ]
 [ 3459.31        9930.01      ]
 [ 3448.03        9958.76      ]
 [ 3437.56        9987.84      ]
 [ 3425.8        10016.39      ]
 [ 3415.49672709 10038.96198891]
 [ 3404.29873453 10066.06146015]
 [ 3394.8064204  10086.99860199]
 [ 3385.29613986 10100.87727709]]
[[ 3471.23  9901.52]
 [ 3459.31  9930.01]
 [ 3448.03  9958.76]
 [ 3437.56  9987.84]
 [ 3425.8  10016.39]
 [ 3412.79 10044.81]
 [ 3401.45 10072.79]
 [ 3388.63 10101.66]
 [ 3380.8  10122.83]
 [ 3369.52 10151.58]]
13.017117389649126
(5, 2)
Processed trajectory number :  5 out of  586  trajectories
3165.4624999999996
[[ 3162.07       10195.53      ]
 [ 3164.68       10199.46      ]
 [ 3167.3        10203.4       ]
 [ 3167.8        10207.31      ]
 [ 3168.65       10212.04      ]
 [ 3169.44989683 10219.86553136]
 [ 3169.88592878 10228.33537999]
 [ 3171.46812821 10237.31880556]
 [ 3173.61916484 10249.28115281]]
[[ 3162.07 10

In [0]:
#linnear predictor
def linnear_pred(vec, n_pred):
  #find acceleration

  step = (vec[len(vec)-1]-vec[0])/(len(vec)-1)
  traj_total = np.zeros((len(vec)+n_pred,2))
  traj_total[len(traj_total)-n_pred: len(traj_total)]=  np.array([vec[len(vec)-1]+step*i for i in range(1, n_pred+1)])
  traj_total[0:len(vec)] = vec
  return traj_total

data_loader = DataLoader(1, sample_args.pred_length + sample_args.obs_length, list(range(90,100)),force_preprocessing= True, testing = True)

# Maintain the total_error until now
total_error_linnear = 0.
counter = 0.
j = 0
trajs_l = []
info_l =[]
for b in range(data_loader.num_batches):
    # Get the source, target data for the next batch
    x, y, mean_std = data_loader.next_batch()
    mean_x = mean_std[0][0]
    std_x = mean_std[0][1]
    mean_y = mean_std[1][0]
    std_y = mean_std[1][1]
    traj_real = x[0]
    
    indexes = mean_std[2]
    
    # The observed part of the trajectory
    obs_traj = x[0][:sample_args.obs_length]
    
    # Get the complete trajectory with both the observed and the predicted part from the model
    complete_traj =linnear_pred(obs_traj,sample_args.pred_length)
    
    trajs_l.append(complete_traj)
    info_l.append(mean_std)
    j+=1
    obs_traj = np.array(obs_traj)
    complete_traj[:,0] = complete_traj[:,0] *(std_x + 0.5) +mean_x
    complete_traj[:,1] = complete_traj[:,1] *(std_y + 0.5) + mean_y
    
    traj_real[:,0] = traj_real[:,0] *(std_x + 0.5) + mean_x
    traj_real[:,1] = traj_real[:,1] *(std_y + 0.5) + mean_y
    
   
    # Compute the mean error between the predicted part and the true trajectory
    error = get_mean_error(complete_traj, traj_real, sample_args.obs_length)
    total_error_linnear += get_mean_error(complete_traj, traj_real, sample_args.obs_length)
    #print("Processed trajectory number : ", b, "out of ", data_loader.num_batches, " trajectories")
    print("Processed trajectory number : ", b, "out of ", data_loader.num_batches, " trajectories")

    if(j <10): #for the first 10 test partitions, let´s see the predicted trajectory, the real one, and the error
      print(mean_x)
      print(complete_traj)
      print(map_data_split[indexes[0]+90, indexes[1]][indexes[2]])
      print(error)
# Print the mean error across all the batches
print("Total mean error of the model is ", total_error_linnear/data_loader.num_batches)

Creating pre-processed data from raw data
Processed trajectory number :  0 out of  586  trajectories
3188.0552978536757
[[ 3159.04370528 10474.6001755 ]
 [ 3178.63540151 10429.59645951]
 [ 3197.65260414 10385.365235  ]
 [ 3216.88948048 10341.43685603]
 [ 3235.86855835 10298.28985586]
 [ 3255.07477162 10254.21227595]
 [ 3274.28098488 10210.13469604]
 [ 3293.48719815 10166.05711614]
 [ 3312.69341141 10121.97953623]]
[[ 3159.04370528 10474.6001755 ]
 [ 3178.63540151 10429.59645951]
 [ 3197.65260414 10385.365235  ]
 [ 3216.88948048 10341.43685603]
 [ 3235.86855835 10298.28985586]
 [ 3254.10794472 10256.27910802]
 [ 3272.8876781  10214.07059312]
 [ 3291.54139771 10172.55610703]
 [ 3305.94341512 10139.88822281]
 [ 3323.01729004 10099.92607869]]
8.094895795426513
Processed trajectory number :  1 out of  586  trajectories
3371.5125
[[ 3392.9    10089.47  ]
 [ 3378.61   10119.87  ]
 [ 3363.23   10152.57  ]
 [ 3351.31   10182.29  ]
 [ 3339.25   10211.94  ]
 [ 3325.8375 10242.5575]
 [ 3312.425  1

In [0]:

np.save('/content/gdrive/My Drive/vanilla18/info', info)
np.save('/content/gdrive/My Drive/vanilla18/info_l', info_l)
np.save('/content/gdrive/My Drive/vanilla18/trajs', trajs)
np.save('/content/gdrive/My Drive/vanilla18/trajs_l', trajs_l)

In [0]:
#Future: 

#We now have a working LSTM model. 

#LSTM social: 
#There is not a lot of social interactions in the dataset...

#Get to work socal LSTM, with social tensor = 0 (just with trajectory data it should work)
#Use social LSTM with social tensor. Probably results wont really improve


#Note LSTM model works slightly worse (almost identically) as the linnear predictor.
#the reason behind this could be the trajectory of the instances in the dataset largely depends on linnear behaviours


